<a href="https://colab.research.google.com/github/svperbeast/ai_lab/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## FCFS(First Come First Served) CPU Scheduling

In [22]:
import copy
import matplotlib.pyplot as plt

class Process:
  def __init__(self, id, requested_n_cores, burst_time):
    self.id = id
    self.requested_n_cores = requested_n_cores
    self.burst_time = burst_time
    self.response_time = 0
    self.return_time = 0

  def __str__(self):
    s = ('Process id {} requested_n_cores {} burst_time {} response_time {} ' +
        'return_time {}')
    return s.format(self.id, self.requested_n_cores, self.burst_time,
                    self.response_time, self.return_time)

class FCFS:
  def __init__(self, n_cores):
    self.n_cores = n_cores

  def run(self, input_procs):
    if len(input_procs) == 0:
      raise Exception('the number of input_procs is 0.')

    procs = copy.copy(input_procs)
    handled_procs = []

    core_status = [0] * self.n_cores
    cur_time = 0
    
    # handle the first process
    proc = procs[0]
    n_cores = proc.requested_n_cores
    if n_cores > self.n_cores:
      ncores = self.n_cores

    proc.return_time = proc.burst_time
    proc.response_time = 0

    for i in range(len(core_status)):
       core_status[i] += proc.return_time

    procs.remove(proc)
    handled_procs.append(proc)

    while len(procs) > 0:
      proc = procs[0]

      n_cores = proc.requested_n_cores
      if n_cores > self.n_cores:
        ncores = self.n_cores
      
      n_avail_cores = []
      for i in range(len(core_status)):
        if core_status[i] <= cur_time:
          n_avail_cores.append(i)
      
      if n_cores <= len(n_avail_cores):
        for ci in n_avail_cores:
          core_status[ci] += proc.return_time

        proc.return_time = cur_time + proc.burst_time
        proc.response_time = cur_time

        procs.remove(proc)
        handled_procs.append(proc)
      else:
        cur_time += max(core_status)
        print('update cur_time: {}'.format(cur_time))

    print('done!')
    return handled_procs

In [23]:
sched = FCFS(8)
procs = []
procs.append(Process(1, 1, 100))
procs.append(Process(2, 8, 400))

ret = sched.run(procs)
for r in ret:
  print(r)

update cur_time: 100
done!
Process id 1 requested_n_cores 1 burst_time 100 response_time 0 return_time 100
Process id 2 requested_n_cores 8 burst_time 400 response_time 100 return_time 500
